In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import truncnorm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit

from pysim import SimBoostRegressor, SimBoostClassifier

## Regression

```python
from pysim import SimBoostRegressor
clf = SimBoostRegressor(n_estimators, val_ratio=0.2, degree=2, knot_num=20, reg_lambda=0.1, reg_gamma=10, random_state=0)
```

In [2]:
def data_generator1_reg(random_state=0):
    
    np.random.seed(random_state)
    # data generation
    beta1 = np.array([.2, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0])
    beta2 = np.array([0, .2, 0.3, 0.5, 0, 0, 0, 0, 0, 0])
    beta3 = np.array([0, 0, 0.2, 0.3, 0.5, 0, 0, 0, 0, 0])
    
    beta = np.vstack([beta1,beta2,beta3])
    model_list = [lambda x: 0.2*np.exp(-4*x), lambda x: 3*x**2, lambda x: 2.5*np.sin(1.5 * np.pi*x)]
    
    x = truncnorm.rvs(a=-3,b=3,loc = 0, scale=1/3, size=(20000,10),random_state=random_state)
    noise = np.random.randn(20000).reshape(-1, 1)
    y = np.reshape(0.2 * np.exp(-4 * np.dot(x, beta1)) + \
                   3 * (np.dot(x, beta2))**2 + 2.5 * np.sin(np.pi * 1.5 * np.dot(x, beta3)), [-1, 1]) + noise
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=random_state)
    return beta, model_list, train_x, test_x, train_y, test_y

In [3]:
beta, model_list, train_x, test_x, train_y, test_y = data_generator1_reg()

clf = SimBoostRegressor(n_estimators=5, knot_num=20, reg_lambda=0.05, reg_gamma=10)

clf.fit(train_x, train_y)
clf.visualize()
mean_squared_error(test_y, clf.predict(test_x))

NameError: name 'importance_ratios_temp' is not defined

In [4]:
x = train_x
 clf.val_auc_ = []
clf.estimators_ = []
component_importance_temp = {}
clf.component_importance_ = {}
pred_val = clf.dummy_intercept_ + np.zeros(len(clf.val_idx))
proba_val = 1 / (1 + np.exp(- pred_val.ravel()))
val_mse = roc_auc_score(y[clf.val_idx], pred_val)

for indice, est in enumerate(clf.sim_estimators_):
    component_importance_temp.update({"sim " + str(indice + 1): {"type": "sim", "indice": indice,
                                          "importance": np.std(est.predict(x[clf.tr_idx, :]))}})

for indice, est in enumerate(clf.dummy_estimators_):
    feature_name = list(est.named_steps.keys())[0]
    component_importance_temp.update({feature_name: {"type": "dummy_lr", "indice": indice,
                                      "importance": np.std(est.predict(x[self.tr_idx, :]))}})


NameError: name 'y' is not defined

In [ ]:
clf.visualize_val_perf()

## LogitBoost

```python
from pysim import SimBoostClassifier
clf = SimBoostClassifier(n_estimators, val_ratio=0.2, degree=2, knot_num=20, reg_lambda=0.1, reg_gamma=10, random_state=0)
```

In [ ]:
def data_generator1_cls(random_state=0):
    
    np.random.seed(random_state)
    # data generation
    beta1 = np.array([.2, 0.3, 0.5, 0, 0, 0, 0, 0, 0, 0])
    beta2 = np.array([0, .2, 0.3, 0.5, 0, 0, 0, 0, 0, 0])
    beta3 = np.array([0, 0, 0.2, 0.3, 0.5, 0, 0, 0, 0, 0])
    
    beta = np.vstack([beta1,beta2,beta3])
    model_list = [lambda x: 0.2*np.exp(-4*x), lambda x: 3*x**2, lambda x: 2.5*np.sin(1.5 * np.pi*x)]
    
    x = truncnorm.rvs(a=-3, b=3, loc = 0, scale=1/3, size=(10000, 10),random_state=random_state)
    noise = np.random.randn(10000).reshape(-1, 1)
    y = np.reshape(0.2 * np.exp(-4 * np.dot(x, beta1)) + \
                   3 * (np.dot(x, beta2))**2 + 2.5 * np.sin(np.pi * 1.5 * np.dot(x, beta3)), [-1, 1]) + noise
    y[y <= 0] = 0
    y[y > 0] = 1
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=random_state)
    return beta, model_list, train_x, test_x, train_y, test_y

In [ ]:
beta, model_list, train_x, test_x, train_y, test_y = data_generator1_cls()

clf = SimBoostClassifier(n_estimators=5, knot_num=20, reg_lambda=0.05, reg_gamma=10)
clf.fit(train_x, train_y)
clf.visualize()
roc_auc_score(test_y, clf.predict_proba(test_x))